# Лабораторная работа №8

## Рекуррентные нейронные сети для анализа временных рядов

Набор данных для прогнозирования временных рядов, который состоит из среднемесячного числа пятен на солнце, наблюдаемых с января 1749 по август 2017.

Данные в виде csv-файла можно скачать на сайте _Kaggle_: https://www.kaggle.com/robervalt/sunspots/

### Задание 1

Загрузите данные. Изобразите ряд в виде графика. Вычислите основные характеристики временного ряда (сезонность, тренд, автокорреляцию).

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [0]:
BASE_DIR = '/content/drive/My Drive/Colab Files/mo-2'

import sys

sys.path.append(BASE_DIR)

import os

In [0]:
DATA_ARCHIVE_NAME = 'sunspots.zip'

LOCAL_DIR_NAME = 'sunspots'

In [0]:
from zipfile import ZipFile

with ZipFile(os.path.join(BASE_DIR, DATA_ARCHIVE_NAME), 'r') as zip_:
    zip_.extractall(LOCAL_DIR_NAME)

In [0]:
DATA_FILE_PATH = 'sunspots/Sunspots.csv'

In [0]:
import pandas as pd

all_df = pd.read_csv(DATA_FILE_PATH)

In [7]:
print(all_df.shape)

(3252, 3)


### Задание 2

Для прогнозирования разделите временной ряд на обучающую, валидационную и контрольную выборки.

In [0]:
train_df = all_df.sample(frac = 0.5)
all_df = all_df.drop(train_df.index)

val_df = all_df.sample(frac = 0.5)
test_df = all_df.drop(val_df.index)

In [9]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(1626, 3)
(813, 3)
(813, 3)


### Задание 3

Примените модель _ARIMA_ для прогнозирования значений данного временного ряда.

### Задание 4

Повторите эксперимент по прогнозированию, реализовав рекуррентную нейронную сеть (с как минимум 2 рекуррентными слоями).

### Задание 5

Сравните качество прогноза моделей.

Какой максимальный результат удалось получить на контрольной выборке?